In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3 as sql

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sql-murder-mystery-database/sql-murder-mystery.db


In [6]:
#Setting up a connection to the database
con = sql.connect('../input/sql-murder-mystery-database/sql-murder-mystery.db')


In [14]:
# List all tables in the database
query_1 = '''
SELECT name 
  FROM sqlite_master
 where type = 'table';
'''
pd.read_sql_query(query_1, con)

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


We will start from the first table to view any report on murder

In [15]:
#Query for crime scene report table
query_2 = '''
SELECT *
FROM crime_scene_report
WHERE city = "SQL City"
ORDER BY date;
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_2, con)

,date,type,description,city
0,20170712,theft,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hiding, no escape. In the distance, the beast\n falters, tethered to the void. The killing blow comes without\n hesitation, without mercy.",SQL City
1,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and future.",SQL City
2,20171110,robbery,"The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardians who fell at the Twilight Gap. Gifted\n to the survivors of that terrible battle, the Gjallarhorn\n is seen as a symbol of honor and survival.",SQL City
3,20180103,bribery,"Apparently, Cayde thought it necessary to expose this extremely\n rare vegetable to a Hive summoning ritual.",SQL City
4,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!",SQL City
5,20180115,assault,Report Not Found,SQL City
6,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City
7,20180215,murder,REDACTED REDACTED REDACTED,SQL City
8,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City


6th field report: 
> Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".

Need to find the witness

In [19]:
#Query for person table to find 1st witness - last house number
query_3 = '''
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1;
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_3, con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [20]:
#Query for person table to find 2nd witness - last hourse number
query_4 = '''
SELECT *
FROM person
WHERE address_street_name = 'Franklin Ave'
AND name LIKE '%Annabel%';
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_4, con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Found the 2 witnesses. Let's check their statement on the crime

In [26]:
#Query for interview table
query_5 = '''
SELECT *
FROM interview
WHERE person_id = 14887 OR person_id = 16371;
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_5, con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Killer details mentioned:
* A male with a gun
* Get Fit Now Gym
* gym membership number starts with "48Z"
* Had gold members bags
* Car with a plate that included "H42W"
* Seen in gym on 9th Jan - same time as Annabel

Based on the data, the tables to check - get_fit_now_member, get_fit_now_check_in and drivers_license

First fetching Annabel's data

In [31]:
#Query for gym members table - Annabel
query_6 = '''
SELECT *
FROM get_fit_now_member
WHERE person_id = 16371;
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_6, con)

,id,person_id,name,membership_start_date,membership_status
0,90081,16371,Annabel Miller,20160208,gold


In [32]:
#Query for gym checkin table - 9th jan - Annabel
query_7 = '''
SELECT *
FROM get_fit_now_check_in
WHERE membership_id = '90081';
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_7, con)

,membership_id,check_in_date,check_in_time,check_out_time
0,90081,20180109,1600,1700


Now finding suspects in the gym

In [28]:
#Query for gym members table - male and gold members with id
query_8 = '''
SELECT *
FROM get_fit_now_member
WHERE membership_status = 'gold' 
AND id LIKE '48Z%';
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_8, con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


Two suspects identified. Checking who visited on 9th Jan 

In [30]:
#Query for gym checkin table - 9th jan - Annabel
query_9 = '''
SELECT *
FROM get_fit_now_check_in
WHERE membership_id = '48Z7A' OR membership_id = '48Z55';
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_9, con)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


Both have timing overlap with Annabel.
Finding license id now 

In [37]:
#Query for person table - 2 suspects
query_10 = '''
SELECT *
FROM person
WHERE id IN (28819, 67318);
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_10, con)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [38]:
#Query for drivers_license table 
query_11 = '''
SELECT *
FROM drivers_license
WHERE id IN (173289, 423327);
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_11, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


**The killer is Jeremy Bowers**

But there is a twist, should check his interview on the crime

In [39]:
#Query for interview table
query_12 = '''
SELECT *
FROM interview
WHERE person_id = 67318 ;
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_12, con)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


New killer:
* A Woman 
* Has a lot of money
* Around 5'5" (65") or 5'7" (67")
* Red hair 
* drives a Tesla Model S
* Attended the SQL Symphony Concert 3 times in December 2017  

In [45]:
#Query for facebook_event_checkin table
query_13 = '''
SELECT person_id, COUNT(*) AS attended_times
FROM facebook_event_checkin
WHERE event_name = 'SQL Symphony Concert'
AND date LIKE '201712%'
GROUP BY person_id
HAVING attended_times = 3 ;
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_13, con)

,person_id,attended_times
0,24556,3
1,99716,3


Two suspects. Now to check their appearance and car

In [46]:
#Query for person table
query_13 = '''
SELECT *
FROM person
WHERE id IN (24556,99716);
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_13, con)

,id,name,license_id,address_number,address_street_name,ssn
0,24556,Bryan Pardo,101191,703,Machine Ln,816663882
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [47]:
#Query for drivers_license table 
query_14 = '''
SELECT *
FROM drivers_license
WHERE id IN (101191, 202298);
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running the query
pd.read_sql_query(query_14, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S


**The killer is Miranda Priestly**